# Running the file<br>
If you wish to use your own copy of the data, use the following command:<br>
<br>
``python lending-club-main.py [{-i |--input=}<loan-csv>] [-h | --help]``<br>
<br>
Here are some examples:<br>
<br>
``python lending-club-main.py --input=loan.csv``<br>
``python lending-club-main.py -i loan.csv``<br>
``python lending-club-main.py``<br>
``python lending-club-main.py --help``<br>
<br>
All of these arguments are optional. Providing no arguments makes the code read from the default location, i.e. ```./data```.<br>
<br>
# Instructions on regenerating this Jupyter Notebook<br>
The Jupyter notebook can be regenerated by installing P2J, like so:<br>
<br>
``pip install p2j``<br>
<br>
and running the following:<br>
<br>
``p2j -o code/lending-club-main.py -t notebook/lending-club-main.ipynb``

# Library Imports

In [ ]:
import getopt
import logging
import sys

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Constants<br>
A bunch of constants are set up so that strings don't clutter the source everywhere.

In [ ]:
DEFAULT_DATASET_LOCATION = "../data"
DEFAULT_LOAN_CSV_FILENAME = "loan.csv"

In [ ]:
ID = "id"
MEMBER_ID = "member_id"
DELINQUENT_2_YEARS = 'delinq_2yrs'
EARLIER_CREDIT_LINE = 'earliest_cr_line'
LAST_PAYMENT_AMOUNT = 'last_pymnt_amnt'
LAST_CREDIT_PULL_DATE = 'last_credit_pull_d'
APPLICATION_TYPE = 'application_type'
LAST_PAYMENT_DATE = 'last_pymnt_d'
COLLECTION_RECOVERY_FEE = 'collection_recovery_fee'
RECOVERIES = 'recoveries'
TOTAL_RECOVERED_LATE_FEE = 'total_rec_late_fee'
TOTAL_PAYMENT = 'total_pymnt'
TOTAL_PAYMENT_INVESTED = 'total_pymnt_inv'
TOTAL_RECOVERED_PRINCIPAL = 'total_rec_prncp'
OUT_PRINCIPAL_INVESTED = 'out_prncp_inv'
OUT_PRINCIPAL = 'out_prncp'
NUM_INQUIRIES_6_MONTHS = 'inq_last_6mths'
NUM_OPRN_CREDIT_LINES = 'open_acc'
NUM_DEROGATORY_PUBLIC_RECORDS = 'pub_rec'
TOTAL_CREDOT_REVOLVING_BALANCE = 'revol_bal'
REVOLVING_LINE_UTILISATION_RATE = 'revol_util'
CURRENT_NUM_CREDIT_LINES = 'total_acc'
INTEREST_RECEIVED_TILL_DATE = 'total_rec_int'
PAYMENT_PLAN = 'pymnt_plan'
INITIAL_LIST_STATUS = 'initial_list_status'
POLICY_CODE = 'policy_code'
URL = 'url'
EMPLOYMENT_TITLE = 'emp_title'
FUNDED_AMOUNT = 'funded_amnt'
FUNDED_AMOUNT_INVESTED = 'funded_amnt_inv'
INSTALLMENT = "installment"
LOAN_STATUS = 'loan_status'
INTEREST_RATE = 'int_rate'
EMPLOYMENT_LENGTH = 'emp_length'
ISSUE_DATE = 'issue_d'
ISSUE_MONTH = 'issue_month'
ISSUE_YEAR = 'issue_year'
MONTHS_SINCE_LAST_DELINQUENCY = "mths_since_last_delinq"
MONTHS_SINCE_LAST_RECORD = "mths_since_last_record"
NEXT_PAYMENT_DATE = "next_pymnt_d"
LOAN_AMOUNT = 'loan_amnt'
TERM = 'term'
INTEREST_RATE_CATEGORY = 'int_rate_cat'
GRADE = 'grade'
SUB_GRADE = 'sub_grade'
HOME_OWNERSHIP = 'home_ownership'
ANNUAL_INCOME = 'annual_inc'
VERIFICATION_STATUS = 'verification_status'

In [ ]:
FULLY_PAID = 'Fully Paid'
CURRENT = 'Current'
CONSTANT_VALUED_COLUMNS = [PAYMENT_PLAN, INITIAL_LIST_STATUS, POLICY_CODE, EMPLOYMENT_TITLE, URL]
CUSTOMER_BEHAVIOUR_COLUMNS = [DELINQUENT_2_YEARS, EARLIER_CREDIT_LINE, NUM_INQUIRIES_6_MONTHS, NUM_OPRN_CREDIT_LINES,
                              NUM_DEROGATORY_PUBLIC_RECORDS,
                              TOTAL_CREDOT_REVOLVING_BALANCE, REVOLVING_LINE_UTILISATION_RATE, CURRENT_NUM_CREDIT_LINES,
                              OUT_PRINCIPAL, OUT_PRINCIPAL_INVESTED, TOTAL_PAYMENT, TOTAL_PAYMENT_INVESTED,
                              TOTAL_RECOVERED_PRINCIPAL,
                              INTEREST_RECEIVED_TILL_DATE, TOTAL_RECOVERED_LATE_FEE, RECOVERIES,
                              COLLECTION_RECOVERY_FEE, LAST_PAYMENT_DATE,
                              LAST_PAYMENT_AMOUNT, LAST_CREDIT_PULL_DATE, APPLICATION_TYPE,
                              MONTHS_SINCE_LAST_DELINQUENCY, MONTHS_SINCE_LAST_RECORD, NEXT_PAYMENT_DATE]

# Cleaning Loan Data<br>
The cleanup consists of the following steps:<br>
- Removing `desc` column<br>
- Removing `member_id` column<br>
- Removing completely null columns<br>
- Removing customer behaviour columns which are not available for analysis for aspiring loan applicants<br>
- Removing columns which have constant values and do not contribute to analysis

In [ ]:
def non_null_loans(raw_loans):
    ## Removing Desc column from dataset as it will be not helpful for us in this case study, whereas it can be helpful if we were solving NLP problem
    loans_wo_desc = raw_loans.drop('desc', axis=1)
    # Checking which column can be used as an identifier
    heading("Check which column can be used as an identifier")
    verify_id_member_id_columns_are_not_correlated(loans_wo_desc)
    loans_wo_desc_member_id = loans_wo_desc.drop(columns=[MEMBER_ID], axis=1)
    # Both this column can be used as an identifier, anyone of these can be dropped. Also none of this is helpful for our analysis. They are just identifier
    heading("Column Data Types")
    logging.debug(loans_wo_desc_member_id.dtypes)
    loans_wo_nulls = clean_null_columns(loans_wo_desc_member_id)
    heading("Loan Info after scrubbing completely empty columns")
    logging.debug(loans_wo_nulls.info())
    loans_wo_nulls = clean_customer_behaviour_columns(loans_wo_nulls)
    loans_wo_nulls = clean_constant_valued_columns(loans_wo_nulls)
    return loans_wo_nulls

## Null Column Cleanup<br>
The loan dataset has several columns which are completely empty. These are useless for analysis.<br>
This function drops completely null columns.

In [ ]:
def clean_null_columns(loans):
    heading("Null Entries Statistics")
    null_entry_statistics = loans.isnull().sum() / len(loans.index)
    null_columns = null_entry_statistics[null_entry_statistics == 1.0].index.to_numpy()
    heading("Completely Null Columns")
    logging.info(null_columns)
    return loans.drop(null_columns, axis=1)

Verifying that member_id's and id's are completely unique from each other

In [ ]:
def verify_id_member_id_columns_are_not_correlated(loans):
    ids_as_number = pd.to_numeric(loans[ID])
    member_ids_as_number = pd.to_numeric(loans[MEMBER_ID])
    logging.info(f"Unique entries in id column : {ids_as_number.nunique()}")
    logging.info(f"Unique entries in member_id column : {member_ids_as_number.nunique()}")
    ids_not_in_member_ids = set(ids_as_number).difference(set(member_ids_as_number))
    member_ids_not_in_ids = set(member_ids_as_number).difference(set(ids_as_number))
    logging.info(f"id's not in member_id's = {len(ids_not_in_member_ids)}")
    logging.info(f"member_id's not in id's = {len(member_ids_not_in_ids)}")

## Dropping Customer Behaviour Columns<br>
Removing columns  which are customer behaviour variable. This values will not be available to us while customer is filling the loan form.<br>
Hence this will be not helpful for deciding whether customer will charged off or full pay

In [ ]:
def clean_customer_behaviour_columns(loans_wo_nulls):
    return loans_wo_nulls.drop(columns=CUSTOMER_BEHAVIOUR_COLUMNS, axis=1)

## Constant-Valued Columns<br>
There are a few columns which have constant values. Such as pymnt_plan,initial_list_status,policy_code. We can drop these. Along with this, we can also drop<br>
emp_title and URL column because they too don't contain values which can help do decide loan_status

In [ ]:
def clean_constant_valued_columns(loans_wo_nulls):
    logging.info(f"Unique Values in column pymnt_plan: {loans_wo_nulls[PAYMENT_PLAN].nunique()}")
    logging.info(f"Unique Values in column initial_list_status: {loans_wo_nulls[INITIAL_LIST_STATUS].nunique()}")
    logging.info(f"Unique Values in column policy_code: {loans_wo_nulls[POLICY_CODE].nunique()}")
    return loans_wo_nulls.drop(columns=CONSTANT_VALUED_COLUMNS, axis=1)

# Checking Correlation between columns<br>
We check the correlation between pairs of columns, this gives us a statistic (the default being the Pearson correlation coefficient).<br>
The correlation coefficient gives a measure of how one factor varies with the other.

In [ ]:
def multicollinear_free_loads(loans):
    heading("Check for Highly Correlated variables")
    correlations = loans.corr()
    indexes = correlations.index.to_numpy()
    heading("Correlation Coefficients")
    logging.info(correlations.to_string())
    heading("Highly Correlated Columns")
    for row in indexes:
        for col in indexes:
            if (row == col):
                continue
            if (correlations[row][col] >= 0.85):
                logging.info(f"[{row},{col}] = {correlations[row][col]}")

    # loan_amnt, funded_amnt and funded_amnt_inv have high correlation. High correlation means they all contains the same information. We can drop 2 out of 3.
    # we will keep loan_amnt and rest 2 can be dropped.
    # installment is also highly correlated
    return loans.drop(columns=[FUNDED_AMOUNT, FUNDED_AMOUNT_INVESTED], axis=1)

# Remove loans which are of status CURRENT<br>
Loans with `CURRENT` status are current customers involved in paying off their loans.<br>
These loans do not provide us with any training information from which we can infer any conclusions.

In [ ]:
def loans_without_current_loans(loans):
    heading("Loan Statuses: There should only be 3 categories")
    logging.info(loans[LOAN_STATUS].value_counts())
    print("Shape of the data before dropping rows", loans.shape)
    loans_wo_current = loans[loans[LOAN_STATUS] != CURRENT]
    print("Shape of the data after dropping rows", loans_wo_current.shape)
    return loans_wo_current

# Correction of Data Types<br>
Some columns needed for numerical EDA do not show up as numbers, or properly-formatted dates.<br>
Examples are: for `interest rate`, `employment length` and `issue date`. We fix these factors here.

In [ ]:
def corrected_data_types(loans):
    loans = loans.copy()
    loans[INTEREST_RATE] = loans[INTEREST_RATE].apply(lambda x: float(str(x).replace('%', '')))
    loans[EMPLOYMENT_LENGTH] = loans[EMPLOYMENT_LENGTH].apply(
        lambda x: float(str(x).replace('years', '').replace('year', '').replace('+', '').replace('< 1', '0.5')))
    loans[ISSUE_DATE] = pd.to_datetime(loans[ISSUE_DATE], format='%b-%y')
    loans[ISSUE_MONTH] = pd.DatetimeIndex(loans[ISSUE_DATE]).month
    loans[ISSUE_YEAR] = pd.DatetimeIndex(loans[ISSUE_DATE]).year
    return loans

In [ ]:
def analyse(loans):
    plot_overall_charged_off_vs_paid(loans)
    plot_loan_amount_across_time_by_loan_status(loans)
    for i in loans.select_dtypes(include=['int', 'float']).columns:
        sns.boxplot(data=loans, x=LOAN_STATUS, y=i)
        plt.title("Loan Status vs " + i)
        plt.show()
    plot_requested_loan_amount_by_loan_status(loans)
    plot_users_term_by_loan_status(loans)
    loans[INTEREST_RATE_CATEGORY] = pd.cut(loans[INTEREST_RATE], 4, labels=["low", "med", "high", "vhigh"])
    logging.info(pd.cut(loans[INTEREST_RATE], 4, ).value_counts())
    plot_users_loan_category_by_loan_status(loans)
    plot_installment_by_loan_status(loans)
    loans.groupby(by=[GRADE])[LOAN_STATUS].value_counts().unstack().apply(lambda x: x / sum(x), axis=1).plot(kind='bar',
                                                                                                             stacked=True)
    plot_sub_grade_by_loan_status(loans)
    plot_employment_length_by_loan_status(loans)
    plot_home_ownership_by_loan_status(loans)
    plot_annual_income_by_loan_status(loans)
    logging.info(loans[ANNUAL_INCOME].describe().apply(lambda x: '%.5f' % x))
    plot_restricted_annual_income_by_loan_status(loans)
    plot_verification_status_by_loan_status(loans)

In [ ]:
def plot_sub_grade_by_loan_status(loans):
    plt.figure(figsize=(10, 6))
    loans.groupby(by=[SUB_GRADE])[LOAN_STATUS].value_counts().unstack().apply(lambda x: x / sum(x), axis=1).plot(
        kind='bar', stacked=True);

In [ ]:
def plot_verification_status_by_loan_status(loans):
    loans.groupby(by=[VERIFICATION_STATUS, LOAN_STATUS])[LOAN_STATUS].count().unstack().fillna(0).plot.bar()
    plt.show()

In [ ]:
def plot_restricted_annual_income_by_loan_status(loans):
    sns.boxplot(data=loans[(loans[ANNUAL_INCOME] > 10000) & (loans[ANNUAL_INCOME] < 100000)], y=ANNUAL_INCOME,
                x=LOAN_STATUS)

In [ ]:
def plot_annual_income_by_loan_status(loans):
    sns.boxplot(data=loans, y=ANNUAL_INCOME, x=LOAN_STATUS)

In [ ]:
def plot_home_ownership_by_loan_status(loans):
    loans.groupby(by=[HOME_OWNERSHIP, LOAN_STATUS])[LOAN_STATUS].count().unstack().fillna(0).plot.bar()
    plt.title("Home Ownership")
    plt.show()

In [ ]:
def plot_employment_length_by_loan_status(loans):
    sns.histplot(data=loans, x=EMPLOYMENT_LENGTH, hue=LOAN_STATUS, multiple='stack');

In [ ]:
def plot_installment_by_loan_status(loans):
    plt.figure(figsize=(10, 5))
    plt.subplot(1, 2, 1)
    sns.histplot(data=loans[loans[LOAN_STATUS] == FULLY_PAID], x=INSTALLMENT)
    plt.subplot(1, 2, 2)
    sns.histplot(data=loans[loans[LOAN_STATUS] != FULLY_PAID], x=INSTALLMENT, color='r')
    plt.show()

In [ ]:
def plot_users_loan_category_by_loan_status(loans):
    loans.groupby(by=[INTEREST_RATE_CATEGORY, LOAN_STATUS])[LOAN_STATUS].count().unstack().apply(
        lambda x: x / sum(x),
        axis=1).plot(
        kind='bar', stacked=True)
    plt.show()

In [ ]:
def plot_requested_loan_amount_by_loan_status(loans):
    plt.figure(figsize=(10, 5))
    sns.histplot(data=loans, x=LOAN_AMOUNT, hue=LOAN_STATUS, binwidth=1000, multiple="stack")
    plt.title("Loan Amount Requested Distribution")
    plt.xlabel("Loan Amount Requested")
    plt.show()

In [ ]:
def plot_users_term_by_loan_status(loans):
    plt.figure(figsize=(10, 8))
    loans.groupby(by=[TERM, LOAN_STATUS])[LOAN_STATUS].count().unstack().apply(lambda x: x / sum(x), axis=1).plot(
        kind='bar', stacked=True)
    plt.title("Distribution of users for loan term")

In [ ]:
def plot_loan_amount_across_time_by_loan_status(loans):
    plt.figure(figsize=(20, 10))
    sns.lineplot(data=loans, x=ISSUE_DATE, y=LOAN_AMOUNT, hue=LOAN_STATUS)
    plt.title("Loan Given across the date")
    plt.xticks(rotation=90)
    plt.show()

In [ ]:
def plot_overall_charged_off_vs_paid(loans):
    loans[LOAN_STATUS].value_counts(1).plot(kind='bar')
    plt.show()

In [ ]:
def study(raw_loans):
    logging.debug(raw_loans.head().to_string())
    # Number of Rows and Columns in the data set
    logging.debug(raw_loans.shape)
    logging.debug(raw_loans.columns)
    loans_wo_unneeded_columns = non_null_loans(raw_loans)
    loans_wo_correlated_factors = multicollinear_free_loads(loans_wo_unneeded_columns)
    loans_wo_current = loans_without_current_loans(loans_wo_correlated_factors)
    loans_with_corrected_data_types = corrected_data_types(loans_wo_current)
    recheck_null_values(loans_with_corrected_data_types)
    loans_with_corrected_data_types.set_index(keys=ID, inplace=True)
    analyse(loans_with_corrected_data_types)

# Recheck Null Values after Cleanup<br>
We still have null values in the data. We can fill those values. But EDA doesn't require to fill. Handling missing value is part of modelling.

In [ ]:
def recheck_null_values(loans_with_corrected_data_types):
    heading("Null value checks after Cleanup")
    logging.info(loans_with_corrected_data_types.isnull().sum())

This function reads the loan data set

In [ ]:
def parse_commandline_options(args):
    print(f"args are: {args}")
    loan_csv = f"{DEFAULT_DATASET_LOCATION}/{DEFAULT_LOAN_CSV_FILENAME}"
    try:
        options, arguments = getopt.getopt(args, "i:hf:", ["input=", "help"])
        for option, argument in options:
            if option in ("-h", "--help"):
                print_help_text()
            elif option in ("-i", "--input"):
                loan_csv = argument
            else:
                print(f"{option} was not recognised as a valid option")
                print_help_text()
                print("Allowing to continue since Jupyter notebook passes in other command-line options")
        return loan_csv
    except getopt.GetoptError as e:
        sys.stderr.write("%s: %s\n" % (args[0], e.msg))
        print_help_text()
        exit(2)

In [ ]:
def print_help_text():
    print("USAGE: python lending-club-main.py [{-i |--input=}<loan-csv>]")

This function overrides Jupyter's default logger so that we can output things based on our formatting preferences

In [ ]:
def setup_logging():
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)
    logger = logging.getLogger()
    formatter = logging.Formatter('%(message)s')
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    ch.setFormatter(formatter)
    logger.setLevel(logging.DEBUG)
    logger.addHandler(ch)

In [ ]:
def read_csv(loan_csv):
    return pd.read_csv(loan_csv, low_memory=False)

In [ ]:
def heading(heading_text):
    logging.info("-" * 100)
    logging.info(heading_text)
    logging.info("-" * 100)

In [ ]:
def main():
    setup_logging()
    study(read_csv(parse_commandline_options(sys.argv[1:])))

In [ ]:
main()